Regression Model to Predict Program Performance Score

In [264]:
import numpy as np
import pandas as pd
import math

from sklearn import linear_model, preprocessing
from sklearn.neural_network import MLPRegressor

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')

/var/folders/w8/rx32_whx4n158n20x1kr4hph0000gn/T/ipykernel_1426/3732955502.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [265]:
x_data = pd.read_csv('x_all.csv')
x = x_data.to_numpy()
x_data

,total-operators,distinct-operators,total-operands,distinct-operands,function-calls,loops,assignments,max-cyclomatic-complexity,sum-cyclomatic-complexity,dynamic-memory-calls,file-access,file-operation,file-position,input-output,optimization
0,643,61,358,209,54,2,13,3,5,4,6,4,5,30,0
1,899,96,471,268,75,3,8,5,8,6,6,4,5,30,0
2,899,96,471,268,75,3,8,5,8,6,6,4,5,30,1
3,899,96,471,268,75,3,8,5,8,6,6,4,5,30,2
4,899,96,471,268,75,3,8,5,8,6,6,4,5,30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,1215,111,637,384,81,5,13,5,10,4,6,4,5,31,1
59,1215,111,637,384,81,5,13,5,10,4,6,4,5,31,2
60,1215,111,637,384,81,5,13,5,10,4,6,4,5,31,3
61,1313,107,694,442,88,4,13,6,11,7,6,4,5,30,0


In [266]:
y_data = pd.read_csv('y_all.csv')
y = y_data.to_numpy()
y_data

,score
0,0.004178
1,0.068652
2,0.605387
3,1.611539
4,2.514857
...,...
58,2218.014101
59,2171.328028
60,2068.807911
61,5.166882


In [278]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

ann = MLPRegressor(
    # solver='lbfgs',
    max_iter=10000,
    # hidden_layer_sizes=[20],
    # alpha=0.001
    learning_rate_init=0.1,
    learning_rate='constant'
)
ann.fit(x_scaled[1:-1], y[1:-1].ravel())
print(ann.score(x_scaled[1:-1], y[1:-1]))

print(ann.predict([x_scaled[0]]))
print(y[0])
print(ann.predict([x_scaled[-1]]))
print(y[-1])

# lr = linear_model.LinearRegression()
# lr.fit(x_scaled[1:-1], y[1:-1])
# print(lr.score(x_scaled[1:-1], y[1:-1]))

# print(lr.predict([x_scaled[0]]))
# print(y[0])
# print(lr.predict([x_scaled[-1]]))
# print(y[-1])


0.9509524464508752
[-0.64122733]
[0.00417832]
[4.44417574]
[2.98841096]
